In [ ]:

import os
import json
import nest_asyncio
import logging
import sys
from datetime import datetime
from typing import Dict, Any, List, Optional
from dataclasses import dataclass, asdict
from collections import defaultdict
import uuid
import asyncio
import re
from difflib import SequenceMatcher
from autogen import AssistantAgent, UserProxyAgent
from langchain_openai import AzureChatOpenAI
from langchain.schema import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain.memory import ConversationBufferMemory as LCConversationBufferMemory
from langchain.memory import ConversationSummaryBufferMemory





In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
logger = logging.getLogger(__name__)

# Apply nest_asyncio for Jupyter compatibility
nest_asyncio.apply()

In [ ]:
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            "api_key": "",
            "base_url": "",
            "api_type": "azure",
            "api_version": "2024-02-01"
        }
    ],
    "temperature": 0.7,
    "timeout": 120,
    "cache_seed": 42
}


In [ ]:

os.makedirs("data/conversation_history", exist_ok=True)
os.makedirs("data/progress_data", exist_ok=True)
os.makedirs("data/student_conversations", exist_ok=True)

In [ ]:

# LLM Configuration
class LLMConfig:
    def __init__(self):
        self.config = {
            "model": AZURE_CONFIG["gpt_deployment"],
            "api_key": AZURE_CONFIG["api_key"],
            "base_url": AZURE_CONFIG["endpoint"],
            "api_version": AZURE_CONFIG["api_version"],
            "temperature": 0.7,
            "max_tokens": 2000,
            "api_type": "azure"
        }
    
    def get_config(self) -> Dict[str, Any]:
        return self.config
    
    def get_autogen_config(self) -> Dict[str, Any]:
        return {
            "model": self.config["model"],
            "api_key": self.config["api_key"],
            "base_url": self.config["base_url"],
            "api_version": self.config["api_version"],
            "api_type": "azure",
            "temperature": self.config["temperature"],
            "max_tokens": self.config["max_tokens"]
        }

In [ ]:


# Agent Configuration
class AgentConfig:
    @staticmethod
    def get_tutor_system_message() -> str:
        return """You are an expert Educational Tutor Agent with deep knowledge across academic subjects. Adapt explanations to the student's grade level, use multiple teaching approaches, provide step-by-step breakdowns, and encourage critical thinking. Evaluate responses, identify knowledge gaps, and suggest practice. Use age-appropriate language, provide examples, and maintain a supportive tone. Personalize responses based on student needs and track progress."""

    @staticmethod
    def get_student_system_message() -> str:
        return """You are a Student Agent representing a learner. Ask clarifying questions, request examples, express learning needs, and engage actively in problem-solving. Share thought processes, seek feedback, and show curiosity. Admit when you don't understand and build on previous knowledge."""

    @staticmethod
    def get_progress_tracker_system_message() -> str:
        return """You are a Progress Tracker Agent. Monitor student performance, identify patterns, and track improvement. Evaluate responses, identify gaps, and provide data-driven insights. Generate detailed, student-friendly progress reports, suggest focus areas, and recommend personalized learning paths based on the provided data."""


In [ ]:

# LLM Provider
class AzureOpenAIProvider:
    def __init__(self, config: Dict[str, Any]):
        self.config = config
        try:
            self.llm = AzureChatOpenAI(
                deployment_name=config["model"],
                temperature=config["temperature"],
                max_tokens=config["max_tokens"],
                openai_api_key=config["api_key"],
                azure_endpoint=config["base_url"],
                openai_api_version=config["api_version"],
                openai_api_type="azure"
            )
        except Exception as e:
            logger.error(f"Failed to initialize AzureChatOpenAI: {str(e)}")
            raise
    
    def get_langchain_llm(self):
        return self.llm

In [ ]:


# Progress Memory
@dataclass
class LearningProgress:
    subject: str
    topic: str
    skill_level: float
    confidence_level: float
    attempts: int
    successful_attempts: int
    last_interaction: str
    improvement_trend: str
    recommended_actions: List[str]

@dataclass
class LearningSession:
    session_id: str
    student_id: str
    start_time: str
    end_time: Optional[str]
    subjects_covered: List[str]
    topics_covered: List[str]
    questions_asked: int
    questions_answered_correctly: int
    engagement_score: float
    session_summary: str

class ProgressMemory:
    def __init__(self, student_id: str, storage_path: str = "data/progress_data"):
        self.student_id = student_id
        self.storage_path = storage_path
        self.progress_data: Dict[str, Dict[str, LearningProgress]] = defaultdict(dict)
        self.learning_sessions: List[LearningSession] = []
        self.analytics: Dict[str, Any] = {
            "total_sessions": 0,
            "total_questions": 0,
            "average_accuracy": 0.0,
            "favorite_subjects": [],
            "challenging_topics": []
        }
        os.makedirs(self.storage_path, exist_ok=True)
        self._load_progress()
    
    def start_session(self, session_id: str) -> LearningSession:
        session = LearningSession(
            session_id=session_id,
            student_id=self.student_id,
            start_time=datetime.now().isoformat(),
            end_time=None,
            subjects_covered=[],
            topics_covered=[],
            questions_asked=0,
            questions_answered_correctly=0,
            engagement_score=0.0,
            session_summary=""
        )
        self.learning_sessions.append(session)
        self.analytics["total_sessions"] += 1
        return session
    
    def end_session(self, session_id: str, session_summary: str):
        for session in self.learning_sessions:
            if session.session_id == session_id:
                session.end_time = datetime.now().isoformat()
                session.session_summary = session_summary
                self._update_analytics()
                self._save_progress()
                break
    
    def update_progress(self, subject: str, topic: str, performance_score: float, confidence_level: float, was_successful: bool):
        if subject not in self.progress_data:
            self.progress_data[subject] = {}
        
        if topic not in self.progress_data[subject]:
            self.progress_data[subject][topic] = LearningProgress(
                subject=subject,
                topic=topic,
                skill_level=0.0,
                confidence_level=0.0,
                attempts=0,
                successful_attempts=0,
                last_interaction=datetime.now().isoformat(),
                improvement_trend='stable',
                recommended_actions=[]
            )
        
        progress = self.progress_data[subject][topic]
        progress.attempts += 1
        if was_successful:
            progress.successful_attempts += 1
        
        weight = 0.3
        progress.skill_level = progress.skill_level * (1 - weight) + performance_score * weight
        progress.confidence_level = progress.confidence_level * (1 - weight) + confidence_level * weight
        progress.last_interaction = datetime.now().isoformat()
        progress.improvement_trend = self._calculate_improvement_trend(progress)
        progress.recommended_actions = self._generate_recommendations(progress)
        self._save_progress()
    
    def update_session(self, session_id: str, subject: str, topic: str, question_asked: bool = False, correct_answer: bool = False):
        for session in self.learning_sessions:
            if session.session_id == session_id:
                if subject not in session.subjects_covered:
                    session.subjects_covered.append(subject)
                if topic not in session.topics_covered:
                    session.topics_covered.append(topic)
                if question_asked:
                    session.questions_asked += 1
                    self.analytics["total_questions"] += 1
                if correct_answer:
                    session.questions_answered_correctly += 1
                session.engagement_score = self._calculate_engagement_score(session)
                self._save_progress()
                break
    
    def get_progress_report(self) -> Dict[str, Any]:
        report = {
            "student_id": self.student_id,
            "generated_at": datetime.now().isoformat(),
            "overall_analytics": self.analytics,
            "subject_progress": {},
            "recent_sessions": [asdict(session) for session in self.learning_sessions[-5:]],
            "recommendations": self._generate_overall_recommendations()
        }
        
        for subject, topics in self.progress_data.items():
            subject_data = {
                "average_skill_level": 0.0,
                "average_confidence": 0.0,
                "topics_count": len(topics),
                "topics_detail": {topic: asdict(progress) for topic, progress in topics.items()}
            }
            total_skill = sum(progress.skill_level for progress in topics.values())
            total_confidence = sum(progress.confidence_level for progress in topics.values())
            if len(topics) > 0:
                subject_data["average_skill_level"] = total_skill / len(topics)
                subject_data["average_confidence"] = total_confidence / len(topics)
            report["subject_progress"][subject] = subject_data
        
        return report
    
    def get_struggling_areas(self) -> List[Dict[str, Any]]:
        struggling_areas = []
        for subject, topics in self.progress_data.items():
            for topic, progress in topics.items():
                if progress.skill_level < 0.6 or progress.confidence_level < 0.5 or progress.improvement_trend == 'declining':
                    struggling_areas.append({
                        "subject": subject,
                        "topic": topic,
                        "skill_level": progress.skill_level,
                        "confidence_level": progress.confidence_level,
                        "trend": progress.improvement_trend,
                        "recommendations": progress.recommended_actions
                    })
        struggling_areas.sort(key=lambda x: x["skill_level"])
        return struggling_areas
    
    def _calculate_improvement_trend(self, progress: LearningProgress) -> str:
        if progress.attempts < 3:
            return 'stable'
        success_rate = progress.successful_attempts / progress.attempts
        return 'improving' if success_rate > 0.8 and progress.skill_level > 0.7 else 'declining' if success_rate < 0.4 or progress.skill_level < 0.3 else 'stable'
    
    def _generate_recommendations(self, progress: LearningProgress) -> List[str]:
        recommendations = []
        if progress.skill_level < 0.4:
            recommendations.extend([f"Focus on fundamentals in {progress.topic}", "Request step-by-step explanations"])
        if progress.confidence_level < 0.5:
            recommendations.extend(["Practice easier problems", "Seek encouragement"])
        if progress.improvement_trend == 'declining':
            recommendations.extend(["Review basics", "Try different approaches"])
        if progress.skill_level > 0.8:
            recommendations.extend([f"Explore advanced {progress.subject} topics", "Apply concepts practically"])
        return recommendations
    
    def _generate_overall_recommendations(self) -> List[str]:
        recommendations = []
        if self.analytics["average_accuracy"] < 0.6:
            recommendations.extend(["Focus on understanding over speed", "Ask clarifying questions"])
        if len(self.analytics["challenging_topics"]) > 3:
            recommendations.extend(["Schedule regular reviews", "Break down complex topics"])
        return recommendations
    
    def _calculate_engagement_score(self, session: LearningSession) -> float:
        questions_factor = min(session.questions_asked * 0.2, 0.5)
        accuracy_factor = session.questions_answered_correctly / max(session.questions_asked, 1) * 0.3
        topics_factor = len(session.topics_covered) * 0.1
        return min(questions_factor + accuracy_factor + topics_factor, 1.0)
    
    def _update_analytics(self):
        total_questions = sum(session.questions_asked for session in self.learning_sessions)
        total_correct = sum(session.questions_answered_correctly for session in self.learning_sessions)
        self.analytics["total_questions"] = total_questions
        self.analytics["average_accuracy"] = total_correct / max(total_questions, 1)
        self.analytics["favorite_subjects"] = [
            subject for subject, data in self.get_progress_report()["subject_progress"].items()
            if data["average_skill_level"] > 0.7
        ]
        self.analytics["challenging_topics"] = [
            f"{area['subject']}: {area['topic']}" for area in self.get_struggling_areas()
        ]
    
    def _save_progress(self):
        filename = f"{self.student_id}_progress.json"
        filepath = os.path.join(self.storage_path, filename)
        data = {
            "student_id": self.student_id,
            "progress_data": {subject: {topic: asdict(progress) for topic, progress in topics.items()} for subject, topics in self.progress_data.items()},
            "learning_sessions": [asdict(session) for session in self.learning_sessions],
            "analytics": self.analytics,
            "last_updated": datetime.now().isoformat()
        }
        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2)
        except Exception as e:
            logger.error(f"Error saving progress: {str(e)}")
    
    def _load_progress(self):
        filename = f"{self.student_id}_progress.json"
        filepath = os.path.join(self.storage_path, filename)
        if os.path.exists(filepath):
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                for subject, topics in data.get("progress_data", {}).items():
                    self.progress_data[subject] = {topic: LearningProgress(**progress) for topic, progress in topics.items()}
                self.learning_sessions = [LearningSession(**session) for session in data.get("learning_sessions", [])]
                self.analytics = data.get("analytics", self.analytics)
            except Exception as e:
                logger.error(f"Error loading progress: {str(e)}")

In [ ]:


# Simplified Conversation Memory
class ConversationMemory:
    def __init__(self, session_id: Optional[str] = None, storage_path: str = "data/student_conversations"):
        self.session_id = session_id or str(uuid.uuid4())
        self.storage_path = storage_path
        self.conversation_history: List[Dict[str, Any]] = []
        self.metadata: Dict[str, Any] = {
            "session_id": self.session_id,
            "created_at": datetime.now().isoformat(),
            "updated_at": datetime.now().isoformat(),
            "message_count": 0,
            "topics_covered": [],
            "subjects_discussed": []
        }
        os.makedirs(self.storage_path, exist_ok=True)
        self._load_conversation()
    
    def add_message(self, role: str, content: str, metadata: Optional[Dict[str, Any]] = None):
        message = {
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat(),
            "message_id": str(uuid.uuid4()),
            "metadata": metadata or {}
        }
        self.conversation_history.append(message)
        self.metadata["message_count"] += 1
        self.metadata["updated_at"] = datetime.now().isoformat()
        if metadata and "subject" in metadata and metadata["subject"] not in self.metadata["subjects_discussed"]:
            self.metadata["subjects_discussed"].append(metadata["subject"])
        if metadata and "topic" in metadata and metadata["topic"] not in self.metadata["topics_covered"]:
            self.metadata["topics_covered"].append(metadata["topic"])
        self._save_conversation()
    
    def get_messages(self, limit: Optional[int] = None) -> List[Dict[str, Any]]:
        return self.conversation_history[-limit:] if limit else self.conversation_history
    
    def _save_conversation(self):
        filename = f"{self.session_id}.json"
        filepath = os.path.join(self.storage_path, filename)
        data = {
            "metadata": self.metadata,
            "conversation_history": self.conversation_history
        }
        try:
            with open(filepath, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2)
        except Exception as e:
            logger.error(f"Error saving conversation: {str(e)}")
    
    def _load_conversation(self):
        filename = f"{self.session_id}.json"
        filepath = os.path.join(self.storage_path, filename)
        if os.path.exists(filepath):
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                    self.metadata = data.get("metadata", self.metadata)
                    self.conversation_history = data.get("conversation_history", [])
            except Exception as e:
                logger.error(f"Error loading conversation: {str(e)}")

In [ ]:


# Educational Tutor Agent
class EducationalTutorAgent(AssistantAgent):
    def __init__(self, name: str = "Educational_Tutor", student_id: str = "default_student", session_id: Optional[str] = None, **kwargs):
        llm_config_manager = LLMConfig()
        llm_config = llm_config_manager.get_autogen_config()
        
        try:
            super().__init__(
                name=name,
                system_message=AgentConfig.get_tutor_system_message(),
                llm_config=llm_config,
                max_consecutive_auto_reply=5,
                human_input_mode="NEVER",
                description="Educational Tutor Agent",
                code_execution_config=False,
                **kwargs
            )
        except Exception as e:
            logger.error(f"Failed to initialize EducationalTutorAgent: {str(e)}")
            raise
        
        self.student_id = student_id
        self.session_id = session_id
        self.llm = AzureOpenAIProvider(llm_config_manager.get_config()).get_langchain_llm()
        self.conversation_memory = ConversationMemory(session_id=session_id)
        self.progress_memory = ProgressMemory(student_id=student_id)
        
    def explain_concept(self, subject: str, topic: str, difficulty_level: str = "medium", learning_style: str = "mixed") -> str:
        logger.info(f"Generating explanation for {subject}: {topic}")
        progress_report = self.progress_memory.get_progress_report()
        subject_progress = progress_report.get('subject_progress', {}).get(subject, {})
        
        prompt = f"""
        Create a comprehensive explanation for {topic} in {subject}.
        Student Context:
        - Difficulty Level: {difficulty_level}
        - Learning Style: {learning_style}
        - Previous Knowledge: {json.dumps(subject_progress, indent=2) if subject_progress else 'No previous data'}
        
        Include:
        1. Clear introduction
        2. Step-by-step breakdown
        3. Examples and applications
        4. Practice questions
        5. Summary
        """
        
        try:
            explanation = self.llm.invoke([SystemMessage(content=prompt)]).content
            self.conversation_memory.add_message("tutor", explanation, {"subject": subject, "topic": topic})
            self.progress_memory.update_progress(subject, topic, 0.5, 0.5, True)
            self.progress_memory.update_session(self.session_id, subject, topic)
            return explanation
        except Exception as e:
            logger.error(f"Error generating explanation: {str(e)}")
            return f"Error generating explanation: {str(e)}"
    
    def create_practice_problems(self, subject: str, topic: str, count: int = 5, difficulty: str = "medium") -> Dict[str, Any]:
        logger.info(f"Generating {count} practice problems for {subject}: {topic}")
        
        prompt = f"""
        Generate {count} practice problems for {topic} in {subject}.
        Requirements:
        - Difficulty: {difficulty}
        - Mix of types: multiple choice, short answer, problem-solving
        - Include solutions and explanations
        
        Format as JSON:
        {{
            "subject": "{subject}",
            "topic": "{topic}",
            "difficulty": "{difficulty}",
            "problems": [
                {{
                    "id": "problem_1",
                    "type": "multiple_choice",
                    "question": "What is the quadratic formula?",
                    "options": ["x = (-b ± √(b²-4ac))/2a", "x = -b/2a", "x = ac/b", "x = b²-4ac"],
                    "correct_answer": "x = (-b ± √(b²-4ac))/2a",
                    "explanation": "The quadratic formula is used to solve equations of the form ax² + bx + c = 0"
                }}
            ]
        }}
        """
        
        try:
            response_text = self.llm.invoke([SystemMessage(content=prompt)]).content
            json_start = response_text.find('{')
            json_end = response_text.rfind('}') + 1
            json_text = response_text[json_start:json_end]
            problems_data = json.loads(json_text)
            self.conversation_memory.add_message("tutor", json.dumps(problems_data, indent=2), {"subject": subject, "topic": topic})
            self.progress_memory.update_session(self.session_id, subject, topic)
            return problems_data
        except Exception as e:
            logger.error(f"Error generating practice problems: {str(e)}")
            raise
    
    def assess_understanding(self, student_response: str, correct_answer: str, topic: str, subject: str) -> Dict[str, Any]:
        logger.info(f"Assessing understanding for {topic}")
        
        prompt = f"""
        Analyze student response for {topic} in {subject}.
        Correct Answer: {correct_answer}
        Student Response: {student_response}
        
        Provide assessment in JSON format:
        {{
            "accuracy_score": 85,
            "understanding_level": "good",
            "feedback": "Good understanding with minor gaps",
            "suggestions": ["Practice more examples", "Review key concepts"]
        }}
        """
        
        try:
            response_text = self.llm.invoke([SystemMessage(content=prompt)]).content
            json_start = response_text.find('{')
            json_end = response_text.rfind('}') + 1
            json_text = response_text[json_start:json_end]
            assessment_data = json.loads(json_text)
            accuracy = assessment_data.get("accuracy_score", 0) / 100
            self.progress_memory.update_progress(subject, topic, accuracy, accuracy, accuracy >= 0.7)
            self.progress_memory.update_session(self.session_id, subject, topic, correct_answer=accuracy >= 0.7)
            return assessment_data
        except Exception as e:
            logger.error(f"Error assessing understanding: {str(e)}")
            raise

# Student Agent
class StudentAgent(UserProxyAgent):
    def __init__(self, name: str = "Student_Learner", student_id: str = "default_student", grade_level: str = "middle_school", **kwargs):
        llm_config_manager = LLMConfig()
        llm_config = llm_config_manager.get_autogen_config()
        
        try:
            super().__init__(
                name=name,
                system_message=AgentConfig.get_student_system_message() + f"\nGrade Level: {grade_level}",
                llm_config=llm_config,
                max_consecutive_auto_reply=3,
                human_input_mode="NEVER",
                description="Student agent",
                code_execution_config=False,
                **kwargs
            )
        except Exception as e:
            logger.error(f"Failed to initialize StudentAgent: {str(e)}")
            raise
        
        self.student_id = student_id
        self.grade_level = grade_level
        self.conversation_memory = ConversationMemory(session_id=f"student_{student_id}")
    
    def ask_question(self, topic: str, question: str, context: str = "", subject: str = "general") -> Dict[str, Any]:
        logger.info(f"Student asking question about {topic}: {question}")
        
        question_data = {
            "topic": topic,
            "question": question,
            "context": context,
            "timestamp": datetime.now().isoformat()
        }
        
        self.conversation_memory.add_message("student", f"Question about {topic}: {question}", {"subject": subject, "topic": topic})
        return question_data

# Progress Tracker Agent
class ProgressTrackerAgent(AssistantAgent):
    def __init__(self, name: str = "Progress_Tracker", student_id: str = "default_student", **kwargs):
        llm_config_manager = LLMConfig()
        llm_config = llm_config_manager.get_autogen_config()
        
        try:
            super().__init__(
                name=name,
                system_message=AgentConfig.get_progress_tracker_system_message(),
                llm_config=llm_config,
                max_consecutive_auto_reply=2,
                human_input_mode="NEVER",
                description="Progress tracking agent",
                code_execution_config=False,
                **kwargs
            )
        except Exception as e:
            logger.error(f"Failed to initialize ProgressTrackerAgent: {str(e)}")
            raise
        
        self.student_id = student_id
        self.progress_memory = ProgressMemory(student_id=student_id)
        self.llm = AzureOpenAIProvider(llm_config_manager.get_config()).get_langchain_llm()
    
    def generate_progress_report(self) -> Dict[str, Any]:
        logger.info("Generating progress report")
        progress_report = self.progress_memory.get_progress_report()
        
        prompt = f"""
        You are a Progress Tracker Agent tasked with generating a detailed, student-friendly progress report based on the following data:
        {json.dumps(progress_report, indent=2)}
        
        Analyze the data and create a report that includes:
        - A summary of the student's overall progress, highlighting key achievements.
        - A list of strengths, based on high-performing subjects or topics (skill level > 0.7).
        - Areas for improvement, based on struggling topics (skill level < 0.6 or declining trend).
        - Specific, actionable recommendations for future learning.
        
        Format the response as JSON:
        {{
            "summary": "Detailed summary of student progress",
            "strengths": ["Strength 1", "Strength 2"],
            "areas_for_improvement": ["Topic 1", "Topic 2"],
            "recommendations": ["Recommendation 1", "Recommendation 2"],
            "timestamp": "{datetime.now().isoformat()}"
        }}
        
        Ensure the report is accurate, reflects the provided data, and uses positive, encouraging language.
        """
        
        try:
            response_text = self.llm.invoke([SystemMessage(content=prompt)]).content
            json_start = response_text.find('{')
            json_end = response_text.rfind('}') + 1
            json_text = response_text[json_start:json_end]
            report_data = json.loads(json_text)
            return report_data
        except Exception as e:
            logger.error(f"Error generating progress report: {str(e)}")
            raise

# Main Execution
async def main():
    logger.info("Starting educational session")
    print("Starting educational session...", flush=True)
    
    student_id = "test_student_001"
    session_id = str(uuid.uuid4())
    
    try:
        # Initialize agents
        tutor = EducationalTutorAgent(name="Tutor", student_id=student_id, session_id=session_id)
        student = StudentAgent(name="Student", student_id=student_id, grade_level="high_school")
        tracker = ProgressTrackerAgent(name="Progress_Tracker", student_id=student_id)
        tutor.progress_memory.start_session(session_id)
        print("Agents initialized successfully!", flush=True)
        
        # Tutor explains a concept
        explanation = tutor.explain_concept(
            subject="mathematics",
            topic="quadratic equations",
            difficulty_level="medium",
            learning_style="visual"
        )
        print("\n=== EXPLANATION ===", flush=True)
        print(explanation, flush=True)
        
        # Student asks a question
        question_result = student.ask_question(
            topic="quadratic equations",
            question="Can you show a real-world example?",
            context="Learned quadratic formula",
            subject="mathematics"
        )
        tutor.progress_memory.update_session(session_id, "mathematics", "quadratic equations", question_asked=True)
        print("\n=== STUDENT QUESTION ===", flush=True)
        print(json.dumps(question_result, indent=2), flush=True)
        
        # Generate practice problems
        problems = tutor.create_practice_problems(
            subject="mathematics",
            topic="quadratic equations",
            count=3,
            difficulty="medium"
        )
        print("\n=== PRACTICE PROBLEMS ===", flush=True)
        print(json.dumps(problems, indent=2), flush=True)
        
        # Simulate student assessment
        assessment = tutor.assess_understanding(
            student_response="The quadratic formula is x = (-b ± √(b²-4ac))/2a",
            correct_answer="x = (-b ± √(b²-4ac))/2a",
            topic="quadratic equations",
            subject="mathematics"
        )
        print("\n=== ASSESSMENT ===", flush=True)
        print(json.dumps(assessment, indent=2), flush=True)
        
        # End session
        session_summary = "Student engaged with quadratic equations, asked questions, and completed practice problems."
        tutor.progress_memory.end_session(session_id, session_summary)
        
        # Generate progress report
        progress_report = tracker.generate_progress_report()
        print("\n=== PROGRESS REPORT ===", flush=True)
        print(json.dumps(progress_report, indent=2), flush=True)
        
    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}")
        print(f"Error: {str(e)}", flush=True)
    
    logger.info("Educational session completed")
    print("\nEducational session completed.", flush=True)

In [ ]:


# Execute
if __name__ == "__main__":
    os.environ["AUTOGEN_USE_DOCKER"] = "False"
    
    try:
        asyncio.run(main())
    except Exception as e:
        logger.error(f"Error running main: {str(e)}")
        print(f"Error: {str(e)}", flush=True)

INFO:__main__:Starting educational session
Starting educational session...
Agents initialized successfully!
INFO:__main__:Generating explanation for mathematics: quadratic equations
INFO:httpx:HTTP Request: POST https://idkrag.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2025-01-01-preview "HTTP/1.1 200 OK"

=== EXPLANATION ===
### Comprehensive Explanation of Quadratic Equations

---

#### 1. **Introduction: What Are Quadratic Equations?**

A **quadratic equation** is a type of mathematical equation that involves a variable raised to the power of 2 (i.e., squared). The general form of a quadratic equation is:

\[
ax^2 + bx + c = 0
\]

Here:
- \(x\) is the variable.
- \(a\), \(b\), and \(c\) are constants (numbers).
- \(a \neq 0\) (because if \(a = 0\), the equation becomes linear, not quadratic).

Quadratic equations are important because they show up in many real-world scenarios, such as calculating areas, projectile motion, and optimizing profit or costs. 